# Find out WDI - GW (gapminder world) mapping

So we have

- the WDI dataset you made
- the Gapminder World dataset you made

Quite some of the indicators in GW come from an old version of WDI

We want to update the GW indicators with the new WDI data

What do we need to do?

1) Create a file, which contains mapping from WDI id to GW id (e.g. EG.USE.PCAP.KG.OE -> energy_production_total)

For each GW indicator, follow the indicator url to the google spreadsheet (see below how to download xls version). In that spreadsheet, go to the settings tab and get the WDI identifier from the source_link. OR in the about tab Link to complete reference. Only do this if it's a WDI link (e.g. http://data.worldbank.org/indicator/EG.USE.PCAP.KG.OE).

I am not sure if all indicators spreadsheets have the same layout, so program defensively. 

Maybe it's good to extend ddf--concepts of GW with columns source_link for every indicator and wdi_id for wdi indicators, so you can quickly see if you're not missing anything. We might want to import more concept properties from the different tabs of the spreadsheets later on.

2) Check with your new WDI DDF dataset if the WDI id's actually still exist. Add this information to the mapping file.

E.g. This WDI indicator does not exist in the current WDI dataset
https://docs.google.com/spreadsheets/u/1/d/13-1Egm0NN3aQUnontyJBo3rbqQLpiqqVL7vokagPil8/pub#

This WDI indicator does still exist in the current WDI dataset
https://docs.google.com/spreadsheets/u/1/d/1StdAfQCYzvpSYQMvLfXkZ1FMKqcRh6BbZ6B-bLARIz8/pub#

p.s. you can get an xls version of google spreadsheet very easily, just add output=xls as a parameter of the source_link
https://docs.google.com/spreadsheet/pub?key=0AkBd6lyS3EmpdFNZMXZwcjNPY2c3MWwxbWIwVFgyd0E&output=xls
all xls's can also be found in the waffle-server-importers-exporters repo, named by the key parameter in the source_link (e.g. https://docs.google.com/spreadsheet/pub?key=0AkBd6lyS3EmpdFNZMXZwcjNPY2c3MWwxbWIwVFgyd0E = 0AkBd6lyS3EmpdFNZMXZwcjNPY2c3MWwxbWIwVFgyd0E.xls)
https://github.com/Gapminder/waffle-server-importers-exporters/tree/world-legacy-with-data/data/tmp/excel

3) We don't know yet how many WDI indicators don't exist anymore. So let's find that out first and we'll have to decide what to do with the non-existing ones (this is up to other people in Gapminder, we can propose solutions though of course ; ))

4) Once we have done 3, we'll start setting up this merging script I told you about before.

In [123]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
import requests

In [121]:
def to_concept_id(s, sub='[/ -\.\*";]+', sep='_'):
    '''convert a string to lowercase alphanumeric + underscore id for concepts'''
    if s is np.nan:
        return s

    s1 = re.sub(sub, sep, s.strip())
    s1 = s1.replace('\n', '')

    # remove the first/last underscore
    if s1[-1] == sep:
        s1 = s1[:-1]
    if s1[0] == sep:
        s1 = s1[1:]

    return s1.lower()

In [3]:
wdi_path = '../../ddf--wdi/'
gd_path = 'output/ddf/ddf--concepts.csv'

In [4]:
wdi = pd.read_csv(os.path.join(wdi_path, 'output', 'ddf--concepts--continuous.csv'))

In [189]:
gd = pd.read_csv(gd_path)

In [6]:
gd.columns

Index(['concept', 'name', 'concept_type', 'description', 'indicator_url',
       'unit', 'interpolation', 'drill_up', 'domain', 'scales'],
      dtype='object')

In [7]:
wdi.columns

Index(['concept', 'concept_type', 'series_code', 'topic', 'indicator_name',
       'short_definition', 'long_definition', 'unit_of_measure', 'periodicity',
       'base_period', 'other_notes', 'aggregation_method',
       'limitations_and_exceptions', 'notes_from_original_source',
       'general_comments', 'source', 'statistical_concept_and_methodology',
       'development_relevance', 'related_source_links', 'other_web_links',
       'related_indicators', 'license_type'],
      dtype='object')

In [7]:
gd.indicator_url.dropna().ix[40]

'http://spreadsheets.google.com/pub?key=phAwcNAVuyj1gkNuUEXOGag'

In [9]:
url = gd.indicator_url.dropna().ix[40] + '&output=xls'

In [10]:
r = requests.get(url)

In [6]:
from io import BytesIO

In [15]:
sheet = pd.read_excel(BytesIO(r.content), sheetname='Settings')

In [16]:
sheet

,Indicator-settings in the graph,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,Source name,CDIAC (Carbon Dioxide Information Analysis Cen...,NaN,Required! Text that will be shown next to the ...
2,Source link,http://cdiac.ornl.gov/trends/emis/meth_reg.html,NaN,"Link for target, when clicking source name in ..."
3,Scale type,lin,NaN,"Required! Type ""lin"" for linear scale or ""log""..."


In [18]:
sheet = sheet.set_index('Indicator-settings in the graph')
sheet.ix['Source link'].iloc[0]

'http://cdiac.ornl.gov/trends/emis/meth_reg.html'

In [7]:
from tomorrow import threads

In [207]:
# @threads(3, timeout=100)
def get_indicator_url(sheet_url):
    if sheet_url is np.nan:
        return np.nan
    if 'gapminder.org' in sheet_url:
        return np.nan
    sheet_url = sheet_url.replace('%0A', '')
    r = requests.get(sheet_url+'&output=xls')
    return r


def get_sheet(r):
    if r is np.nan:
        return r
    
    try:
        sheet = pd.read_excel(BytesIO(r.content), sheetname='Settings')
        if 'Indicator-settings in the graph' in sheet.columns:
            sheet = sheet.set_index('Indicator-settings in the graph')
            return sheet.ix['Source link'].iloc[0]
        elif 'Not in use' in sheet.columns:
            sheet = sheet.set_index('Not in use')
            return sheet.ix['Link next to axis'].iloc[0]
        elif 'This tab contains settings for the software' in sheet.columns:
            sheet = sheet.set_index('This tab contains settings for the software')
            return sheet.ix['Axis link (where you come when clicking on name above)'].iloc[0]
        else:
            print('format not supported:', r.url)
            print('columns:', sheet.columns)
            return r.url
    except Exception as e:
        print(r.url)
        print(e)
        return r.url

In [208]:
res = []
for i, row in gd[['concept', 'indicator_url']].iterrows():
    concept = row['concept']
    url = row['indicator_url']
    try:
        u = get_indicator_url(url)
    except Exception as e:
        print(e)
        print('error getting xls: ' + url)
        res.append((concept, url))
    res.append((concept, u))

res = dict(res)
for k, v in res.items():
    res[k] = get_sheet(res[k])

error getting xls: http://spreadsheets.google.com/pub?key=tQR7RhlZdPjBkVCDPPF4zUg
https://docs.google.com/spreadsheet/pub?key=0ArfEDsV3bBwCdEV1RkJqTEItQnJYVXJlZzVuc3Y3Mmc%0A%0A&output=xls
Unsupported format, or corrupt file: Expected BOF record; found b'<!DOCTYP'


In [209]:
res_df = pd.DataFrame([res.keys(), res.values()])

In [210]:
res_df = res_df.T

In [211]:
res_df.ix[561][1]

'http://pwt.econ.upenn.edu/php_site/pwt71/pwt71_form.php'

In [212]:
ttt = res_df[res_df[1].str.contains('output=xls').fillna(False)]

*Note: some files says 'Not in use' in the settings tab.*

https://docs.google.com/spreadsheets/d/1fCnUUj9nVbxv5dZJIMiUIhschS59DcYkM3Km1yhRuEo/edit#gid=3



In [213]:
for i, u in ttt[1].items():
    print(u)
#     r = requests.get(u)
#     try:
#         xls = pd.read_excel(BytesIO(r.content), sheetname='Settings')
#         print(xls.head(2).iloc[:, :2])
#     except Exception as e:
#         print('Error', e)
        

https://docs.google.com/spreadsheet/pub?key=0ArfEDsV3bBwCdEV1RkJqTEItQnJYVXJlZzVuc3Y3Mmc%0A%0A&output=xls


In [215]:
ttt

,0,1
12,residential_energy_use_percent,https://docs.google.com/spreadsheet/pub?key=0A...


In [90]:
res_df[res_df[1].str.contains('worldbank.org/indicator/').fillna(False)]

,0,1
16,primary_school_completion_percent_of_boys,http://data.worldbank.org/indicator/SE.PRM.CMP...
17,gnipercapita_ppp_current_international,http://data.worldbank.org/indicator/NY.GNP.PCA...
18,co2_intensity_of_economic_output_kg_co2_per_20...,http://data.worldbank.org/indicator/EN.ATM.CO2...
29,investments_percent_of_gdp,http://data.worldbank.org/indicator/NE.GDI.TOT...
40,population_growth_annual_percent,http://data.worldbank.org/indicator/SP.POP.GROW
53,children_out_of_school_primary_male,http://data.worldbank.org/indicator/SE.PRM.UNE...
66,improved_sanitation_overall_access_percent,http://data.worldbank.org/indicator/SH.STA.ACSN
70,arms_imports_us_inflation_adjusted,http://data.worldbank.org/indicator/MS.MIL.MPR...
74,expenditure_per_student_tertiary_percent_of_gd...,http://data.worldbank.org/indicator/SE.XPD.TER...
82,births_attended_by_skilled_health_staff_percen...,http://data.worldbank.org/indicator/SH.STA.BRT...


In [216]:
res_df_bak = res_df.copy()

In [139]:
res_df = res_df_bak.copy()

In [217]:
res_df.columns = ['concept', 'settings_link']
res_df = res_df.dropna(how='any')

In [163]:
def wdi_id_from_setting_link(s):
    if 'data.worldbank.org/indicator/' in s: 
        if re.match('.*\/([A-Z.]+)\/?.*', s):
            return re.match('.*\/([A-Z.]+)\/?.*', s).groups()[0]
    
    return np.nan

In [218]:
res_df['wdi_id'] = res_df['settings_link'].apply(wdi_id_from_setting_link)

In [219]:
res_df = res_df.set_index('concept')

In [220]:
gd = gd.set_index('concept')

In [221]:
gd['wdi_id'] = res_df['wdi_id']
gd['settings_link'] = res_df['settings_link']

In [222]:
len(gd['wdi_id'].dropna())

86

In [223]:
gd['wdi_id'] = gd['wdi_id'].apply(to_concept_id)

In [224]:
found = []
not_found = []

for i, v in gd['wdi_id'].dropna().items():
    if v in wdi.concept.values:
        found.append(v)
    else:
        not_found.append(v)

In [225]:
found

['dt_oda_odat_kd',
 'dt_oda_odat_pc_zs',
 'dt_oda_odat_gi_zs',
 'dt_dod_dect_gn_zs',
 'fi_res_totl_dt_zs',
 'ny_gdp_mktp_kd',
 'ny_gdp_pcap_kd',
 'ny_gdp_pcap_kd_zg',
 'ny_gnp_mktp_pp_cd',
 'ny_gnp_pcap_kd',
 'ny_gnp_pcap_cd',
 'ny_gnp_pcap_pp_cd',
 'si_pov_gini',
 'ny_gdp_defl_kd_zg',
 'ne_gdi_totl_zs',
 'gc_tax_totl_gd_zs',
 'bx_klt_dinv_wd_gd_zs',
 'bm_klt_dinv_gd_zs',
 'si_pov_dday',
 'nv_agr_totl_zs',
 'nv_ind_totl_zs',
 'nv_srv_tetc_zs',
 'ne_exp_gnfs_zs',
 'ne_imp_gnfs_zs',
 'ms_mil_xprt_kd',
 'ms_mil_mprt_kd',
 'tx_val_tech_mf_zs',
 'tg_val_totl_gd_zs',
 'ne_rsb_gnfs_cd',
 'ne_rsb_gnfs_zs',
 'cm_mkt_lcap_gd_zs',
 'ms_mil_xpnd_gd_zs',
 'se_enr_prsc_fm_zs',
 'se_prm_uner',
 'se_prm_uner_fe',
 'se_prm_uner_ma',
 'se_prm_cmpt_zs',
 'se_prm_cmpt_ma_zs',
 'se_prm_cmpt_fe_zs',
 'se_xpd_prim_pc_zs',
 'se_xpd_seco_pc_zs',
 'se_xpd_tert_pc_zs',
 'eg_use_pcap_kg_oe',
 'eg_use_elec_kh_pc',
 'eg_use_elec_kh_pc',
 'ag_lnd_agri_zs',
 'sh_med_phys_zs',
 'sh_sta_brtc_zs',
 'sp_dyn_conu_zs',
 's

In [226]:
not_found

['si_pov',
 'se_adt',
 'eg_use_comm_kt_oe',
 'eg_egy_prod_kt_oe',
 'eg_egy_prod_kt_oe',
 'eg_elc_hyro_kh',
 'eg_elc_hyro_kh',
 'eg_elc_nucl_kh',
 'eg_elc_nucl_kh',
 'en_atm_co',
 'ag_lnd_frst_k',
 'ag_srf_totl_k',
 'it_net_bbnd_p',
 'it_cel_sets_p',
 'it_net_user',
 'it_net_user_p',
 'is_rod_pave_zs',
 'is_veh_nveh_p']

In [227]:
gd['found_in_wdi'] = gd['wdi_id'].dropna().apply(lambda x: True if x in found else False)

In [229]:
gd[['settings_link', 'wdi_id', 'found_in_wdi']].to_csv('output/tmp/gw_wdi.csv')

In [156]:
!open .

In [179]:
sli = gd[['settings_link', 'wdi_id', 'found_in_wdi']]

In [183]:
sli[sli['settings_link'].str.contains('0ArfEDsV3bBwCdEV1RkJqTEItQnJYVXJlZzVuc3Y3Mmc').fillna(False)]

,settings_link,wdi_id,found_in_wdi
concept,,,
residential_energy_use_percent,https://docs.google.com/spreadsheet/pub?key=0A...,NaN,NaN


In [228]:
gd.loc['residential_energy_use_percent', 'settings_link'] = 'http://data.iea.org/IEASTORE/DEFAULT.ASP'

In [186]:
gd.loc['residential_energy_use_percent']

name                                        Residential energy use
concept_type                                               measure
description      Percent of total emergy consumption that is us...
indicator_url    https://docs.google.com/spreadsheet/pub?key=0A...
unit                                                             %
interpolation                                                  NaN
drill_up                                                       NaN
domain                                                         NaN
scales                                           ['linear', 'log']
wdi_id                                                         NaN
settings_link             http://data.iea.org/IEASTORE/DEFAULT.ASP
found_in_wdi                                                   NaN
Name: residential_energy_use_percent, dtype: object

In [233]:
gd.loc['new_births_total_number_estimated', 'indicator_url']

'https://spreadsheets.google.com/pub?key=0ArfEDsV3bBwCdERQeFplM2VWczVrMTFfMXVrQkJpVXc'

In [234]:
gd.loc['mean_years_in_school_women_25_to_34_years', 'indicator_url']

'http://spreadsheets.google.com/pub?key=0ArfEDsV3bBwCdC1MYzAtY2xPQ2xOR1lMeGhYSWlpR0E'

In [237]:
gd[gd['indicator_url'] == 'http://spreadsheets.google.com/pub?key=tQR7RhlZdPjBkVCDPPF4zUg']

,name,concept_type,description,indicator_url,unit,interpolation,drill_up,domain,scales,wdi_id,settings_link,found_in_wdi
concept,,,,,,,,,,,,
aid_given_percent_of_gni,"Aid given, % of GNI",measure,Percentage of the gross national income (GNI) ...,http://spreadsheets.google.com/pub?key=tQR7Rhl...,NaN,NaN,NaN,NaN,"['linear', 'log']",NaN,http://www.oecd.org/dac/stats/qwids,NaN


In [239]:
wdi['concept']

0          ag_agr_trac_no
1       ag_con_fert_pt_zs
2          ag_con_fert_zs
3          ag_lnd_agri_k2
4          ag_lnd_agri_zs
5          ag_lnd_arbl_ha
6       ag_lnd_arbl_ha_pc
7          ag_lnd_arbl_zs
8          ag_lnd_crel_ha
9          ag_lnd_crop_zs
10         ag_lnd_el5m_zs
11         ag_lnd_frst_k2
12         ag_lnd_frst_zs
13      ag_lnd_irig_ag_zs
14         ag_lnd_prcp_mm
15         ag_lnd_totl_k2
16         ag_lnd_trac_zs
17         ag_prd_crel_mt
18         ag_prd_crop_xd
19         ag_prd_food_xd
20         ag_prd_lvsk_xd
21         ag_srf_totl_k2
22         ag_yld_crel_kg
23      bg_gsr_nfsv_gd_zs
24         bm_gsr_cmcp_zs
25         bm_gsr_fcty_cd
26         bm_gsr_gnfs_cd
27         bm_gsr_insf_zs
28         bm_gsr_mrch_cd
29         bm_gsr_nfsv_cd
              ...        
1315    tx_val_fuel_zs_un
1316    tx_val_ictg_zs_un
1317    tx_val_insf_zs_wt
1318    tx_val_manf_zs_un
1319    tx_val_mmtl_zs_un
1320    tx_val_mrch_al_zs
1321    tx_val_mrch_cd_wt
1322    tx_v